In [ ]:
import boto3
from botocore.config import Config

master = boto3.session.Session(profile_name='xxx')
cost_explorer = master.client('ce')
#cost_explorer = master.resource('c3')

In [ ]:
cost = cost_explorer.get_cost_and_usage(
        TimePeriod={
            'Start': '2021-01-01',
            'End': '2021-12-31'
        },
        Granularity='MONTHLY',
        Metrics=['BlendedCost'],
        GroupBy=[
            {
                'Type': 'DIMENSION',
                'Key': 'LINKED_ACCOUNT'
            },
        ]
    )